In [19]:
%store -r

In [20]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_lol-coach-agent',
  'RoleId': 'AROASLCWLO4GENFDXKKU4',
  'Arn': 'arn:aws:iam::161242380044:role/AmazonBedrockExecutionRoleForAgents_lol-coach-agent',
  'CreateDate': datetime.datetime(2025, 10, 11, 7, 32, 46, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]},
  'MaxSessionDuration': 3600,
  'RoleLastUsed': {'LastUsedDate': datetime.datetime(2025, 10, 11, 16, 38, 24, tzinfo=tzutc()),
   'Region': 'us-east-1'}},
 'ResponseMetadata': {'RequestId': '25e512ea-cadb-4c90-a3fa-a8e82486c620',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 11 Oct 2025 16:49:37 GMT',
   'x-amzn-requestid': '25e512ea-cadb-4c90-a3fa-a8e82486c620',
   'content-type': 'text/xml',
   'content-length': '1006'},
  'RetryAttempts': 0}}

# Setup

In [14]:
from knowledge_base import KnowledgeBasesForAmazonBedrock
from agent import invoke_agent_helper
import boto3
import os
import time
import json
import ipywidgets as widgets

In [3]:
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
s3_client = boto3.client('s3')

In [6]:
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
agent_name = 'lol-coach-agent'

In [15]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        ('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'),
        ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
print(agent_foundation_model_selector)

agent_foundation_model = agent_foundation_model_selector.value
print(agent_foundation_model)

Dropdown(description='FM:', options=(('Claude 3 Sonnet', 'anthropic.claude-3-sonnet-20240229-v1:0'), ('Claude 3 Haiku', 'anthropic.claude-3-haiku-20240307-v1:0')), value='anthropic.claude-3-sonnet-20240229-v1:0')
anthropic.claude-3-sonnet-20240229-v1:0


In [24]:
agent_id = 'UFSZYQKWFU'

# Setting up Knowledge Base Information

In [ ]:
knowledge_base_name = f'{agent_name}-kb'
knowledge_base_description = "Knowledge Base containing information about League of Legends early game impact"
bucket_name = f'{agent_name}-{suffix}'

# Creating Knowledge Base for Amazon Bedrock

In [8]:
kb = KnowledgeBasesForAmazonBedrock()
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(knowledge_base_name, knowledge_base_description, bucket_name)

[2025-10-12 02:13:54,511] p81738 {credentials.py:1224} INFO - Found credentials in environment variables.
[2025-10-12 02:13:57,316] p81738 {credentials.py:1224} INFO - Found credentials in environment variables.


Creating KB lol-coach-agent-kb
Step 1 - Creating or retrieving lol-coach-agent-us-east-1-161242380044 S3 bucket for Knowledge Base documents
Creating bucket lol-coach-agent-us-east-1-161242380044
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_729) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '314',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Sat, 11 Oct 2025 18:14:07 '
                                                 'GMT',
                                         'x-amzn-requestid': '5e6a5be1-777c-42b7-9ff5-f22734c66ae6'},
                        'HTTPStatusCode': 200,
                        'RequestId': '5

[2025-10-12 02:18:44,090] p81738 {base.py:258} INFO - PUT https://spljh3mzj5olfj7zye81.us-east-1.aoss.amazonaws.com:443/lol-coach-agent-kb-index-729 [status:200 request:2.026s]



Creating index:
{ 'acknowledged': True,
  'index': 'lol-coach-agent-kb-index-729',
  'shards_acknowledged': True}
Step 6 - Creating Knowledge Base
{'type': 'VECTOR', 'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}}
{ 'createdAt': datetime.datetime(2025, 10, 11, 18, 19, 45, 215177, tzinfo=tzutc()),
  'description': 'Knowledge Base containing League of Legends information',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-east-1:161242380044:knowledge-base/WKEF55LDYB',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'WKEF55LDYB',
  'name': 'lol-coach-agent-kb',
  'roleArn': 'arn:aws:iam::161242380044:role/AmazonBedrockExecutionRoleForKnowledgeBase_729',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchS

In [9]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("kb_documents", bucket_name)

uploading file kb_documents/league-legends-study.pdf to lol-coach-agent-us-east-1-161242380044


In [11]:
# ensure that the kb is available
i_status = ['CREATING', 'DELETING', 'UPDATING']
while bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)['knowledgeBase']['status'] in i_status:
    time.sleep(10)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

{ 'dataSourceId': 'SOVPB0EAUO',
  'ingestionJobId': 'UW0PH9VDE2',
  'knowledgeBaseId': 'WKEF55LDYB',
  'startedAt': datetime.datetime(2025, 10, 12, 5, 0, 53, 633837, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 10, 12, 5, 0, 53, 633837, tzinfo=tzutc())}
{ 'dataSourceId': 'SOVPB0EAUO',
  'ingestionJobId': 'UW0PH9VDE2',
  'knowledgeBaseId': 'WKEF55LDYB',
  'startedAt': datetime.datetime(2025, 10, 12, 5, 0, 53, 633837, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
           

# Testing Knowledge Base

In [34]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Can you describe the role of a top laner during early game?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, agent_foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The top laner's role in the early game is focused on gaining experience (XP). The study found that the top laner relies heavily on early game XP gained, more so than other roles. This suggests that for top laners, accumulating XP in the first 10 minutes is crucial for their overall performance and impact on the game outcome. The top laner's creep score (CS), which measures the number of minions and monsters killed, is also an important factor in the early game according to the analysis. After XP, the top laner benefits more from a high CS than other roles in the early stages.



# Updating Agent role to allow Knowledge Base Retrieve and Retrieve and Generate queries

In [ ]:
kb_policies_statements = [
    {
        "Sid": "QueryKB",
        "Effect": "Allow",
        "Action": [
            "bedrock:Retrieve",
            "bedrock:RetrieveAndGenerate"
        ],
        "Resource": [
            f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
        ]
    }
]
bedrock_agent_kb_policy_statement = {
    "Version": "2012-10-17",
    "Statement": kb_policies_statements
}
bedrock_agent_kb_policy_json = json.dumps(bedrock_agent_kb_policy_statement)
kb_policy_name = f"{agent_name}-kb-{kb_id}"
agent_kb_policy = iam_client.create_policy(
    PolicyName=kb_policy_name,
    PolicyDocument=bedrock_agent_kb_policy_json
)
iam_client.attach_role_policy(
    RoleName=agent_role['Role']['RoleName'],
    PolicyArn=agent_kb_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '17966220-41aa-4d2c-9b56-bbb67e28c214',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 12 Oct 2025 05:12:45 GMT',
   'x-amzn-requestid': '17966220-41aa-4d2c-9b56-bbb67e28c214',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

# Associate Knowledge Base with Agent

In [25]:
#time.sleep(10)
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about League of Legends early game impact.',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

In [26]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
intermediate_agent_status = ['CREATING', 'PREPARING', 'UPDATING', 'VERSIONING']
while bedrock_agent_client.get_agent(agentId=agent_id)['agent']['agentStatus'] in intermediate_agent_status:
    time.sleep(10)

{'ResponseMetadata': {'RequestId': '7eb472db-5806-46fe-ad5f-5d63703b6f70', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 12 Oct 2025 06:12:33 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '7eb472db-5806-46fe-ad5f-5d63703b6f70', 'x-amz-apigw-id': 'SUgAyFtUIAMEVRw=', 'x-amzn-trace-id': 'Root=1-68eb46d1-36f0292a04340d8356ed2729'}, 'RetryAttempts': 0}, 'agentId': 'UFSZYQKWFU', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 10, 12, 6, 12, 33, 697013, tzinfo=tzutc())}


# Invoking Agent

In [28]:
alias_id = 'TSTALIASID'

In [ ]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "Can you describe the role of a top laner during early game?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)



The study on the early game impact in League of Legends found that a player's performance in the first 10 minutes, measured by factors like experience earned, gold, damage taken, and creep score (CS), has a significant correlation with their overall performance at the end of the game. This correlation was strongest for the Mid and Carry roles, while the Top role was least impacted by early game performance.






The study also examined how individual players' early game metrics correlated with the overall team's end game performance. While there was a correlation, it was weaker than the individual early-to-end game correlation, except for the Top laner's experience and the Mid/Carry's gold income which were important predictors of team performance.






Using early game features, the researchers built a predictive model that could determine the match outcome with 62% accuracy, which is comparable to state-of-the-art for such a complex game. The findings suggest focusing on improvin